# PLEASE REMEMBER TO CLEAR THE GPU MEMORY AFTER EACH STEP!!!!!!

In [1]:
%cd ..

/workspaces/Chatchat_AIMeng/src


/workspaces/Chatchat_AIMeng/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from datasets import load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import torch
import os
## Downloading Model 
from utils.constants import *
from utils.util_funcs import *


/workspaces/Chatchat_AIMeng/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load the dataset
train_tokenized = load_from_disk('data/processed/train_tokenized')
val_tokenized = load_from_disk('data/processed/val_tokenized')
test_tokenized = load_from_disk('data/processed/test_tokenized')

In [4]:

training_args = TrainingArguments(
    output_dir=FINE_TUNED_MODEL_PATH,
    save_total_limit=SAVE_TOTAL_LIMIT,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    evaluation_strategy=EVALUATION_STRATEGY,
)

# Evaluation

## Evaluate the base model

In [5]:
loaded_original_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_PATH)

Initialize the trainer and eval on test dataset

In [7]:
trainer = Trainer(
    model=loaded_original_model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized
)

In [8]:
eval_results = trainer.evaluate(eval_dataset=test_tokenized)

In [9]:
eval_results

{'eval_loss': 39.28157043457031,
 'eval_runtime': 50.671,
 'eval_samples_per_second': 28.063,
 'eval_steps_per_second': 7.026}

In [10]:
del loaded_original_model
clear_gpu_memory()

# After fine tune with 5 epochs

In [14]:
SAVE_MODEL_PATH = os.path.join(MODEL_DIR, 'save')

fintuned_model = AutoModelForSeq2SeqLM.from_pretrained(SAVE_MODEL_PATH)

In [15]:
trainer = Trainer(
    model=fintuned_model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized
)

In [16]:
finetuned_eval_results = trainer.evaluate(eval_dataset=test_tokenized)


In [17]:
finetuned_eval_results

{'eval_loss': 0.16730812191963196,
 'eval_runtime': 49.9755,
 'eval_samples_per_second': 28.454,
 'eval_steps_per_second': 7.123}

This result represents the evaluation of the model before and after fine-tuning. Each dictionary contains the following keys:

- `eval_loss`: The loss value of the model on the evaluation dataset. The lower the loss value, the better the performance of the model.
- `eval_runtime`: The runtime of the evaluation process, in seconds.
- `eval_samples_per_second`: The number of samples processed per second. The higher this value, the faster the model processes.
- `eval_steps_per_second`: The number of steps executed per second. The higher this value, the faster the model processes.
From these results, you can see that the loss value of the model on the evaluation dataset has significantly reduced after fine-tuning, from 39.28 to 0.17, indicating a significant improvement in the performance of the model.

At the same time, the processing speed of the model has also improved after fine-tuning, with eval_samples_per_second increasing from 28.063 to 28.454, and eval_steps_per_second increasing from 7.026 to 7.123. This indicates that the model has also made some improvements in processing speed.

In summary, these results show that fine-tuning has had a positive impact on both the performance and processing speed of the model.

In [18]:
print("before fine tune:",eval_results)
print("==========================Compared==========================")
print("after fine tune:",finetuned_eval_results)

before fine tune: {'eval_loss': 39.28157043457031, 'eval_runtime': 50.671, 'eval_samples_per_second': 28.063, 'eval_steps_per_second': 7.026}
==========================Compared==========================
after fine tune: {'eval_loss': 0.16730812191963196, 'eval_runtime': 49.9755, 'eval_samples_per_second': 28.454, 'eval_steps_per_second': 7.123}


In [19]:
del fintuned_model
clear_gpu_memory()